In [1]:
import requests
import ast
import numpy as np
from math import sqrt
import warnings
import ast
from random import gauss, seed
import pulp

warnings.simplefilter("ignore")

In [2]:
url = "https://covid19api.io/api/v1/ReportsByCountries/US"
response = requests.request("GET", url)
output = ast.literal_eval(response.text)

total = output['report']['active_cases'][0]['currently_infected_patients']
mild = output['report']['active_cases'][0]['inMidCondition']
critical = output['report']['active_cases'][0]['criticalStates']

print (output['report']['active_cases'][0])

{'currently_infected_patients': 247868, 'inMidCondition': 178911, 'criticalStates': 68957}


In [3]:
ventilators = int(input("Please enter the number of ventilators Available:"))

Please enter the number of ventilators Available:100000


In [4]:
# Since we have patients in critical condition, we will be providing ventilators to them first
remaining = ventilators-critical
if remaining>0:
    print('Remaining ventilators:', remaining)
else:
    print('All the ventilators have been used up. Possible deaths:', mild-remaining)

Remaining ventilators: 31043


In [5]:
if remaining>0:
    age_groups = [1,11,21,31,41,51,61,71,81]
    percents = [.0364, .0806, .2040, .2366, .1669, .1439, .0959, .0301, 0.004]
    fatality = [0, 0.2, 0.2, 0.2, 0.4, 1.3, 3.6, 8, 14.8]
    survWvent = [.77, .73, .67, .63, .58, .23, .21, .16, .16]
    survWO = [.4, .35, .35, .3, .25, .12, .15, .08, .08]
    num_cases = [int(mild*i) for i in percents]    
    deaths = [int(num_cases[i]*fatality[i]) for i in range(len(fatality))]
    life_remaining = [78.54-i-5.5 for i in age_groups]
    life = [int(x) if x>=0 else 0 for x in life_remaining]
    ventilator_usage = [0,0,0,0,0,0,0,0,0]
    
    url = "https://covid19api.io/api/v1/FatalityRateByAge"
    response = requests.request("GET", url)
    output = ast.literal_eval(response.text)
    for i in output['table']:
        print (i['Age'], i['DeathRateAllCases'])

0 - 17  years old 0.04%
18 - 44 years old 4.5%
45 - 64 years old 23.1%
65 - 74 years old 24.6%
75+ years old 47.7%
TOTAL 100%


In [6]:
print ("Patients in each age group")
print ("O - 10 years: " + str(round(num_cases[0])) + " patients")
print ("11 - 20 years: " + str(round(num_cases[1])) + " patients")
print ("21 - 30 years: " + str(round(num_cases[2])) + " patients")
print ("31 - 40 years: " + str(round(num_cases[3])) + " patients")
print ("41 - 50 years: " + str(round(num_cases[4])) + " patients")
print ("51 - 60 years: " + str(round(num_cases[5])) + " patients")
print ("61 - 70 years: " + str(round(num_cases[6])) + " patients")
print ("71 - 80 years: " + str(round(num_cases[7])) + " patients")
print ("81+     years: " + str(round(num_cases[8])) + " patients")

Patients in each age group
O - 10 years: 6512 patients
11 - 20 years: 14420 patients
21 - 30 years: 36497 patients
31 - 40 years: 42330 patients
41 - 50 years: 29860 patients
51 - 60 years: 25745 patients
61 - 70 years: 17157 patients
71 - 80 years: 5385 patients
81+     years: 715 patients


In [7]:
x = 0

seed(1)
for _ in range(int(remaining)):
    value = gauss(55, 15)
    x = x + 1
   
        
    if value < 11:
        ventilator_usage[0] = ventilator_usage[0] + 1
    elif (value < 21):
        ventilator_usage[1] = ventilator_usage[1] + 1
    elif (value < 31):
        ventilator_usage[2] = ventilator_usage[2] + 1
    elif (value < 41):
        ventilator_usage[3] = ventilator_usage[3] + 1
    elif (value < 51):
        ventilator_usage[4] = ventilator_usage[4] + 1
    elif (value < 61):
        ventilator_usage[5] = ventilator_usage[5] + 1
    elif (value < 71):
        ventilator_usage[6] = ventilator_usage[6] + 1
    elif (value < 81):
        ventilator_usage[7] = ventilator_usage[7] + 1
    else:
        ventilator_usage[8] = ventilator_usage[8] + 1
        
print ("Ventilator usage in each age group")
print ("O - 10 years: " + str(round(ventilator_usage[0])) + " patients")
print ("11 - 20 years: " + str(round(ventilator_usage[1])) + " patients")
print ("21 - 30 years: " + str(round(ventilator_usage[2])) + " patients")
print ("31 - 40 years: " + str(round(ventilator_usage[3])) + " patients")
print ("41 - 50 years: " + str(round(ventilator_usage[4])) + " patients")
print ("51 - 60 years: " + str(round(ventilator_usage[5])) + " patients")
print ("61 - 70 years: " + str(round(ventilator_usage[6])) + " patients")
print ("71 - 80 years: " + str(round(ventilator_usage[7])) + " patients")
print ("81+     years: " + str(round(ventilator_usage[8])) + " patients")
# print(x)
totalyears = survWvent[0] * round(life[0],1) * ventilator_usage[0] + survWO[0]*round(life[0],1)\
*(num_cases[0]-ventilator_usage[0]) + survWvent[1]*round(life[1],1) * ventilator_usage[1] + \
survWO[1]*round(life[1],1)*(num_cases[1]-ventilator_usage[1]) + survWvent[2]*round(life[2],1) * ventilator_usage[2]\
+ survWO[2]*round(life[2],1)*(num_cases[2]-ventilator_usage[2]) + survWvent[3]*round(life[3],1) * ventilator_usage[3]\
+ survWO[3]*round(life[3],1)*(num_cases[3]-ventilator_usage[3]) + survWvent[4]*round(life[4],1) * ventilator_usage[4]\
+ survWO[4]*round(life[4],1)*(num_cases[4]-ventilator_usage[4]) + survWvent[5]*round(life[5],1) * ventilator_usage[5]\
+ survWO[5]*round(life[5],1)*(num_cases[5]-ventilator_usage[5]) + survWvent[6]*round(life[6],1) * ventilator_usage[6]\
+ survWO[6]*round(life[6],1)*(num_cases[6]-ventilator_usage[6]) + .16 * ventilator_usage[7]\
+ .08*(num_cases[7]-ventilator_usage[7])  + .16 * ventilator_usage[8] + .08*(num_cases[8]-ventilator_usage[8])

print ("Current life years expected upon random selection of patients using a normal distribution to emulate 'first \
come first serve' policy: "+ str(round(totalyears)))

Ventilator usage in each age group
O - 10 years: 57 patients
11 - 20 years: 309 patients
21 - 30 years: 1311 patients
31 - 40 years: 3810 patients
41 - 50 years: 6701 patients
51 - 60 years: 8183 patients
61 - 70 years: 6191 patients
71 - 80 years: 3184 patients
81+     years: 1297 patients
Current life years expected upon random selection of patients using a normal distribution to emulate 'first come first serve' policy: 2215082


In [8]:
# Using custom written penalty function

eps = np.finfo(np.float64).eps

def xnew(x0,d,alpha):
    '''
        returns x_new given x_old, d and alpha
    '''
    return (np.array(x0,np.float64)+alpha*np.array(d,np.float64))

def armijo(fun,x0,d,dd,alpha,c):
    '''
        Check if Armijo's condition is satisfied
        https://en.wikipedia.org/wiki/Wolfe_conditions
    '''
    return (fun(xnew(x0,d,alpha)) <= fun(x0)+c*alpha*dd)

def jacobian(fun,x0,epsilon=np.sqrt(eps)):
 
    grad = list()
    evpoints = [1,0]
   
    for i in range(len(x0)):
        fvals = list()
        for _x in evpoints:
            x0[i] += _x*epsilon
            fvals += [fun(x0)]
            x0[i] -= _x*epsilon
        grad += [(fvals[1]-fvals[0])/((evpoints[1]-evpoints[0])*epsilon)]
    return (np.array(grad,np.float64).copy())

def gradient(fun,x0):
        g = jacobian(fun,x0,**{'epsilon':sqrt(eps)})
        return (-g, g)
    

def linear_search(fun,x0,d=None,alpha=1,rho=0.6,c=1e-4,max_iter=1e3,**kwargs):
   
    if any(d):
        pass
        dd = np.dot(d,jacobian(fun,x0,**kwargs))     
    else:
        d = -np.array(jacobian(fun,x0,**kwargs),np.float64) # steepest descent step
        dd = np.dot(d,-d) 
    iters = 0
    while not armijo(fun,x0,d,dd,alpha,c) and iters < max_iter:# Armijo's Condition
        alpha = rho*alpha
        iters += 1
    return ({'x':xnew(x0,d,alpha), 'f':fun(xnew(x0,d,alpha)), 'alpha':alpha, 'iterations':iters})

def minimize(fun,x0,threshold=1e-6,**kwargs):

        ls_kwargs = {'alpha':1,'rho':0.5,'c':1e-4,'max_iter':1e3}
        d, g = gradient(fun,x0)
        if 'max_iter' in kwargs:
            max_iter= kwargs['max_iter']
        else:
            max_iter = 1e3
        x = x0
        iters = 0
        while np.dot(g,g) > threshold and iters < max_iter:
            ls = linear_search(fun,x,d,**ls_kwargs)
            alpha = ls['alpha']
            x = xnew(x,d,alpha)
            d, g = gradient(fun,x)
            iters += 1

        return ({'x':x, 'f':fun(x)})
    
def minimize_constrained(fun,x0,g,c,beta,method='penalty',threshold=1e-2):
        '''
            Constrained Optimization of a function
            Arguments:
            fun - function to be minimized
            x0 - Initial values of x0
            g - list of constraints
            c - Initial constraint weight
            beta - (>1) growth of c after each iteration
            threshold - threshold at which to stop the iterations
        '''
        if method == 'penalty':
            P = lambda x : np.sum([np.max([0.,_(x)])**2 for _  in g])
            f = lambda x : fun(x)+c*P(x)
            chck = lambda x : c*P(x)
        elif method == 'log-barrier':
            B = lambda x : -np.sum([np.log(-_(x)) for _ in g]) # g(x) <= 0
            f = lambda x : fun(x)+(1./c)*B(x)
            chck = lambda x : (1./c)*B(x)
        elif method == 'barrier':
            B = lambda x : -np.sum([1./_(x) for _ in g]) # g(x) <= 0
            f = lambda x : fun(x)+(1/c)*B(x)
            chck = lambda x : (1/c)*B(x)   
        else:
            raise Exception('The method ({}) has not been identified.'.format(method))
        x = x0
        iters = 0
        x_values = list()
        print ('c: -',',x:',x,',f(x):', f(x), ',penalty:',chck(x)) # initial values of function
        while chck(x) > threshold:
            sol = minimize(f,x,threshold)
            x = sol['x']
            x_values.append(x)
            print ('c:',c,',x:',x,',f(x):', round(f(x),3),',penalty:',round(chck(x),3)) # initial values of function
            c *= beta
            iters += 1

        return ({'x':list(map(round,x)),'x_values':x_values, 'f':(round(fun(x))),'c':int(c),'iterations':iters})

In [9]:
fun = lambda x: -(survWvent[0] * round(life[0],1)* x[0] + survWO[0]*round(life[0],1)*(num_cases[0]-x[0]) + \
survWvent[1] * round(life[1],1) * x[1] + survWO[1]*round(life[1],1)*(num_cases[1]-x[1]) + \
survWvent[2] * round(life[2],1) * x[2] + survWO[2]*round(life[2],1)*(num_cases[2]-x[2]) + \
survWvent[3] * round(life[3],1) * x[3] + survWO[3]*round(life[3],1)*(num_cases[3]-x[3]) + \
survWvent[4] * round(life[4],1) * x[4] + survWO[4]*round(life[4],1)*(num_cases[4]-x[4]) + \
survWvent[5] * round(life[5],1) * x[5] + survWO[5]*round(life[5],1)*(num_cases[5]-x[5]) + \
survWvent[6] * round(life[6],1) * x[6] + survWO[6]*round(life[6],1)*(num_cases[6]-x[6]) + \
.16 * x[7] + .08*(num_cases[7]-x[7]) + .16 * x[8] + .08*(num_cases[8]-x[8]))

c1 = lambda x: x[0]+x[1]+x[2]+x[3]+x[4]+x[5]+x[6]+x[7]+x[8]-remaining
c2 = lambda x: -x[0]
c3 = lambda x: -x[1]
c4 = lambda x: -x[2]
c5 = lambda x: -x[3]
c6 = lambda x: -x[4]
c7 = lambda x: -x[5]
c8 = lambda x: -x[6]
c9 = lambda x: -x[7]
c10 = lambda x: -x[8]

x0 = [-1, 0, 0, 0, 0, 0, 0, 0, 0]
g = [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10]
c0 = 0.5
beta = 2

result = minimize_constrained(fun, x0, g,c0, beta, method = 'penalty')

c: - ,x: [-1, 0, 0, 0, 0, 0, 0, 0, 0] ,f(x): -2036253.26 ,penalty: 0.5
c: 0.5 ,x: [11085.33984375  9302.484375    5291.52734375  3683.5078125
  1769.85546875   -15.421875     -17.140625     -17.79296875
   -17.80078125] ,f(x): -2707687.05 ,penalty: 814.933
c: 1.0 ,x: [ 1.26532656e+04  1.02913770e+04  4.97942969e+03  2.84866602e+03
  3.14820312e+02 -7.78906250e+00 -8.64062500e+00 -8.95507812e+00
 -8.95898438e+00] ,f(x): -2741066.327 ,penalty: 400.13
c: 2.0 ,x: [ 1.33209932e+04  1.06763799e+04  4.72931445e+03  2.34333301e+03
 -2.35156250e+00 -4.39257812e+00 -4.81445312e+00 -4.96972656e+00
 -4.96972656e+00] ,f(x): -2753566.037 ,penalty: 255.795
c: 4.0 ,x: [ 1.36077026e+04  1.08248975e+04  4.56743018e+03  2.05674609e+03
 -1.09912109e+00 -2.11523438e+00 -2.33007812e+00 -2.41162109e+00
 -2.40917969e+00] ,f(x): -2758176.463 ,penalty: 137.47
c: 8.0 ,x: [ 1.37520168e+04  1.08996252e+04  4.48580542e+03  1.91229126e+03
 -5.90576172e-01 -1.09985352e+00 -1.20556641e+00 -1.24462891e+00
 -1.24584961e

In [10]:
x = list(map(abs, result['x']))
optimal_life_years = abs(result['f'])

In [11]:
print ("Ventilator usage in each age group")
print ("0 - 10 years: " + str(x[0]) + " patients")
print ("11 - 20 years: " + str(x[1]) + " patients")
print ("21 - 30 years: " + str(x[2]) + " patients")
print ("31 - 40 years: " + str(x[3]) + " patients")
print ("41 - 50 years: " + str(x[4]) + " patients")
print ("51 - 60 years: " + str(x[5]) + " patients")
print ("61 - 70 years: " + str(x[6]) + " patients")
print ("71 - 80 years: " + str(x[7]) + " patients")
print ("81 - 90 years: " + str(x[8]) + " patients")

print ("Total life years through optimization of ventilator usage using custom penalty method: " + str(optimal_life_years))


difference = optimal_life_years - totalyears
print ("By optimizing ventilator usage, we were able to increase expected life years by " + str(round(difference)))

Ventilator usage in each age group
0 - 10 years: 13896.0 patients
11 - 20 years: 10974.0 patients
21 - 30 years: 4404.0 patients
31 - 40 years: 1768.0 patients
41 - 50 years: 0.0 patients
51 - 60 years: 0.0 patients
61 - 70 years: 0.0 patients
71 - 80 years: 0.0 patients
81 - 90 years: 0.0 patients
Total life years through optimization of ventilator usage using custom penalty method: 2762823.0
By optimizing ventilator usage, we were able to increase expected life years by 547741.0


In [12]:
#Using pulp library

model = pulp.LpProblem("Life year maximizing problem", pulp.LpMaximize)

A = pulp.LpVariable('0', lowBound=0, cat='Integer')
B = pulp.LpVariable('11', lowBound=0, cat='Integer')
C = pulp.LpVariable('21', lowBound=0, cat='Integer')
D = pulp.LpVariable('31', lowBound=0, cat='Integer')
E = pulp.LpVariable('41', lowBound=0, cat='Integer')
F = pulp.LpVariable('51', lowBound=0, cat='Integer')
G = pulp.LpVariable('61', lowBound=0, cat='Integer')
H = pulp.LpVariable('71', lowBound=0, cat='Integer')
I = pulp.LpVariable('81', lowBound=0, cat='Integer')
V = pulp.LpVariable('Ventilators', upBound=100000, cat='Integer')

model += survWvent[0] * round(life[0],1)* A + survWO[0]*round(life[0],1)*(num_cases[0]-A) + \
survWvent[1] * round(life[1],1) * B + survWO[1]*round(life[1],1)*(num_cases[1]-B) + \
survWvent[2] * round(life[2],1) * C + survWO[2]*round(life[2],1)*(num_cases[2]-C) + \
survWvent[3] * round(life[3],1) * D + survWO[3]*round(life[3],1)*(num_cases[3]-D) + \
survWvent[4] * round(life[4],1) * E + survWO[4]*round(life[4],1)*(num_cases[4]-E) + \
survWvent[5] * round(life[5],1) * F + survWO[5]*round(life[5],1)*(num_cases[5]-F) + \
survWvent[6] * round(life[6],1) * G + survWO[6]*round(life[6],1)*(num_cases[6]-G) + \
.16 * H + .08*(num_cases[7]-H) + .16 * I + .08*(num_cases[8]-I) 

model += A <= num_cases[0]
model += B <= num_cases[1]
model += C <= num_cases[2]
model += D <= num_cases[3]
model += E <= num_cases[4]
model += F <= num_cases[5]
model += G <= num_cases[6]
model += H <= num_cases[7]
model += I <= num_cases[8]
model += A + B + C + D + E + F + G + H + I <= remaining

model.solve()
print (pulp.LpStatus[model.status])

Optimal


In [13]:
print ("Ventilator usage in each age group")
print ("0 - 10 years: " + str(A.varValue) + " patients")
print ("11 - 20 years: " + str(B.varValue) + " patients")
print ("21 - 30 years: " + str(C.varValue) + " patients")
print ("31 - 40 years: " + str(D.varValue) + " patients")
print ("41 - 50 years: " + str(E.varValue) + " patients")
print ("51 - 60 years: " + str(F.varValue) + " patients")
print ("61 - 70 years: " + str(G.varValue) + " patients")
print ("71 - 80 years: " + str(H.varValue) + " patients")
print ("81 - 90 years: " + str(I.varValue) + " patients")

print ("Total life years through optimization of ventilator usage using pulp library: " + str(round(pulp.value(model.objective))))


difference = int(round(pulp.value(model.objective))) - totalyears
print ("By optimizing ventilator usage, we were able to increase expected life years by " + str(round(difference)))

Ventilator usage in each age group
0 - 10 years: 6512.0 patients
11 - 20 years: 14420.0 patients
21 - 30 years: 10111.0 patients
31 - 40 years: 0.0 patients
41 - 50 years: 0.0 patients
51 - 60 years: 0.0 patients
61 - 70 years: 0.0 patients
71 - 80 years: 0.0 patients
81 - 90 years: 0.0 patients
Total life years through optimization of ventilator usage using pulp library: 2717742
By optimizing ventilator usage, we were able to increase expected life years by 502660
